In [0]:
%sql

CREATE CATALOG IF NOT EXISTS btc_m2;

CREATE SCHEMA IF NOT EXISTS btc_m2.bronze;
CREATE SCHEMA IF NOT EXISTS btc_m2.silver;
CREATE SCHEMA IF NOT EXISTS btc_m2.gold;

In [0]:
 %pip install yfinance fredapi


In [0]:
import yfinance as yf
from fredapi import Fred
import pandas as pd
from pyspark.sql.functions import current_timestamp

FRED_API_KEY = 'e2dab77201621b3294d6ef13fc2ef9a9' 
START_DATE = '2020-01-01'

try:
    fred = Fred(api_key=FRED_API_KEY)
except Exception as e:
    raise RuntimeError(f"Failed to initialize FRED API: {e}")

try:
    df_btc_pd = yf.download("BTC-USD", start=START_DATE, auto_adjust=True)

    if isinstance(df_btc_pd.columns, pd.MultiIndex):
        df_btc_pd.columns = df_btc_pd.columns.get_level_values(0)

    df_btc_pd = df_btc_pd.reset_index()
    df_btc_pd['Date'] = df_btc_pd['Date'].dt.date

except Exception as e:
    print(f"Error fetching Bitcoin data: {e}")
    raise e

try:
    df_m2_pd = fred.get_series('M2SL', observation_start=START_DATE).to_frame(name='m2_supply')
    df_m2_pd.index.name = 'date'
    df_m2_pd = df_m2_pd.reset_index()

except Exception as e:
    print(f"Error fetching FRED data: {e}")
    raise e

df_btc_spark = spark.createDataFrame(df_btc_pd).withColumn("ingestion_date", current_timestamp())
df_m2_spark = spark.createDataFrame(df_m2_pd).withColumn("ingestion_date", current_timestamp())

df_btc_spark.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("btc_m2.bronze.btc_raw")
df_m2_spark.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("btc_m2.bronze.liquidity_raw")